## Photometry on MIRI images

In [ ]:
%load_ext autoreload
%autoreload 2

import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
import subprocess
import miri_utils.photometry_tools as phot

from astropy.io import fits
from astropy.wcs import FITSFixedWarning
from astropy.table import Table

warnings.simplefilter("ignore", category=FITSFixedWarning)


cutout_dir = "/home/bpc/University/master/Red_Cardinal/cutouts_phot/"
phot_dir = "/home/bpc/University/master/Red_Cardinal/photometry/"


# Section to obtain modified apertures

Let's inspect Amirs table:

In [ ]:
table_path =  '/home/bpc/University/master/Red_Cardinal/catalogues/Flux_Aperture_PSFMatched_AperCorr_old.fits'

table = Table.read(table_path)
#print(table[:5])
table.info()
print(table.columns)

Now let's try and call the function:

In [ ]:
cutout_dir = "/home/bpc/University/master/Red_Cardinal/cutouts_phot/"
phot_dir = "/home/bpc/University/master/Red_Cardinal/photometry/"

# Get all FITS file paths
fits_files = glob.glob(os.path.join(cutout_dir, '*.fits'))

# Get the basenames of the FITS files
fits_fnames = [os.path.basename(f) for f in fits_files]

adjusted_apertures = []

for fname in fits_fnames:
    id = fname.split('_')[0]
    filter = fname.split('_')[1]
    survey_obs = fname.split('_')[3]
    
    if '003' in survey_obs:
        survey = 'primer'
        obs = '003'
    elif '004' in survey_obs:
        survey = 'primer'
        obs = '004'
    elif 'cweb1' in survey_obs:
        survey = 'cweb'
        obs = '1'
    elif 'cweb2' in survey_obs:
        survey = 'cweb'
        obs = '2'
    elif 'cos3d1' in survey_obs:
        survey = 'cos3d'
        obs = '1'
    elif 'cos3d2' in survey_obs:
        survey = 'cos3d'
        obs = '2'
    else:
        print(f"Unknown survey and/or observation number for galaxy {id}:\n")
        print(survey_obs)
    
    # Call and collect results
    result = phot.adjust_aperture(id, filter, survey, obs, phot_dir, mask_folder='mask_v5', save_plot=True)
    
    if result:
        adjusted_apertures.append(result)

# After loop: create a DataFrame
df_apertures = pd.DataFrame(adjusted_apertures)

# v5 for manually modified ellipse sizes
df_path = '/home/bpc/University/master/Red_Cardinal/photometry/aperture_table_v5.csv'

# (optional) Save to CSV or integrate into photometry table
df_apertures.to_csv(df_path, index=False)

Now we can easily open any given FITS file with its corresponding ellipse region

In [ ]:
# --- Launch DS9 with the MIRI cutout and the overplotted aperture ---
region_dir = "/home/bpc/University/master/Red_Cardinal/photometry/regions/"
cutout_dir = "/home/bpc/University/master/Red_Cardinal/cutouts_phot/"
phot_dir = "/home/bpc/University/master/Red_Cardinal/photometry/"

id = '10245'
filter = 'F770W'
survey_obs = 'primer004'
cutout_path = os.path.join(cutout_dir, f'{id}_{filter}_cutout_{survey_obs}.fits')
reg_path = os.path.join(region_dir, f'{id}_{survey_obs}_aperture.reg')
subprocess.run(["ds9", cutout_path, "-regions", reg_path])


Let's check the table:

In [ ]:
table_path =  '/home/bpc/University/master/Red_Cardinal/Flux_Aperture_PSFMatched_AperCorr_old.fits'

table = Table.read(table_path)
print(table[:5])
table.info()
print(table['Image_Err'].shape)

# This is where we do most of the work

# Section to perform the actual photometry

In [ ]:
# --- Parameters ---
cutouts_folder = "/home/bpc/University/master/Red_Cardinal/cutouts_phot/"
output_folder = '/home/bpc/University/master/Red_Cardinal/photometry/'
aperture_table_small = '/home/bpc/University/master/Red_Cardinal/photometry/aperture_table_small.csv'
aperture_table_big = '/home/bpc/University/master/Red_Cardinal/photometry/aperture_table_big.csv'
aperture_table = '/home/bpc/University/master/Red_Cardinal/photometry/aperture_table_v5.csv'
fig_path = '/home/bpc/University/master/Red_Cardinal/photometry/Plots_MIRI_phot_v5/'
#fig_path = None
os.makedirs(output_folder, exist_ok=True)

# Get all possible F770W files
all_f770w_files = glob.glob(os.path.join(cutouts_folder, f'*F770W*.fits'))

# Group F770W files by galaxy ID and filter
f770w_files = []
galaxy_ids = set([os.path.basename(f).split('_')[0] for f in all_f770w_files])

#galaxy_ids = ['8465', '7922', '9871', '12202', '8843', '7904', '8338', '10021', '10245', '11136', '12340', '20397']


for galaxy_id in galaxy_ids:
    # Find all F770W files for this galaxy ID
    matching_files = [f for f in all_f770w_files if os.path.basename(f).startswith(galaxy_id)]
    
    # Handle special case for galaxy 11853
    if galaxy_id == '11853':
        # Use the cweb2 file if available
        cweb2_files = [f for f in matching_files if 'cweb2' in f.lower()]
        if cweb2_files:
            f770w_files.append(cweb2_files[0])
            continue  # Skip to the next galaxy
    
    # Prioritise PRIMER over COSMOS-Web
    primer_files = [f for f in matching_files if 'primer' in f.lower()]
    cweb_files = [f for f in matching_files if 'cweb' in f.lower()]
    
    if primer_files:
        f770w_files.append(primer_files[0])  # Prefer PRIMER file
    elif cweb_files:
        f770w_files.append(cweb_files[0])  # Use CWEB only if no PRIMER available


# Get all F1000W files
f1000w_files = glob.glob(os.path.join(cutouts_folder, f'*F1000W*.fits'))

# Get all F1800W files
f1800w_files = glob.glob(os.path.join(cutouts_folder, f'*F1800W*.fits'))

# Get all F1800W files
f2100w_files = glob.glob(os.path.join(cutouts_folder, f'*F2100W*.fits'))


# Section to compare my aperture photometry to the photometry of the COSMOS-Web2025 catalogue

1) Perform the photometry again for my data using the updated and corrected photometry
2) Combine the tables into a single FITS output file

In [ ]:
######################################
## STEP 1:   PERFORM PHOTOMETRY     ##
######################################

phot.perform_photometry(f770w_files, aperture_table_big, output_folder, suffix='v4')

phot.perform_photometry(f1800w_files, aperture_table_big, output_folder, suffix='v4')

#####################################
## STEP 2:   COMBINE THE TABLEs    ##
#####################################

fits_table_v4 = f'Flux_Aperture_PSFMatched_AperCorr_MIRI_v4.fits'
f770w_fname = 'phot_table_F770W_v4.csv'
f1800w_fname = 'phot_table_F1800W_v4.csv'

# Now create the combined FITS table
#phot.combine_filter_csv_to_fits(f770w_fname, f1800w_fname, fits_table_name=fits_table_v4)


In [ ]:
from astropy.coordinates import SkyCoord
import astropy.units as u
from astropy.table import hstack

cweb_path = '/home/bpc/University/master/Red_Cardinal/catalogues/COSMOSWeb_reduced.fits'
my_path = '/home/bpc/University/master/Red_Cardinal/photometry/results/Flux_Aperture_PSFMatched_AperCorr_MIRI_v4.fits'
cat_path = '/home/bpc/University/master/Red_Cardinal/catalogues/cat_targets.fits'

my_table = Table.read(my_path, hdu=1)
cosmos_table = Table.read(cweb_path, hdu=1)
my_cat = Table.read(cat_path, hdu=1)
fits.info(my_path)
fits.info(cweb_path)
#print(my_cat.columns)

# Rename ID column to id to match other catalogues
my_table.rename_column('ID', 'id')

# Reduce catalogue
my_cat_small = my_cat['id', 'ra', 'dec']

# Force type setting
my_table['id'] = my_table['id'].astype(str)
my_cat_small['id'] = my_cat_small['id'].astype(str)

# Match according to IDs
matched = join(my_table, my_cat_small, keys=('id'), join_type='inner')

print(matched.columns)

matched.write('/home/bpc/University/master/Red_Cardinal/catalogues/Flux_Aperture_PSFMatched_AperCorr_MIRI_ra_dec.fits', overwrite=True)

# Build coordinates
coords_my = SkyCoord(ra=matched['ra']*u.deg, dec=matched['dec']*u.deg)
coords_cosmos = SkyCoord(ra=cosmos_table['ra']*u.deg, dec=cosmos_table['dec']*u.deg)

# Match (within 0.3 arcsec, for instance)
idx, d2d, _ = coords_my.match_to_catalog_sky(coords_cosmos)
match_mask = d2d < 0.3 * u.arcsec

# Build matched table
my_matched = matched[match_mask]    # important to take the matched catalogue!
cosmos_matched = cosmos_table[idx[match_mask]]

# Combine tables: rename columns to avoid name collision
cosmos_matched.rename_columns(
    cosmos_matched.colnames,
    [name + "_cosmos" if name in my_matched.colnames else name for name in cosmos_matched.colnames]
)

# Merge horizontally
sky_matched = hstack([my_matched, cosmos_matched])

print(sky_matched.columns)
sky_matched.info()

sky_matched.write('/home/bpc/University/master/Red_Cardinal/catalogues/COSMOSWeb_sky_matched.fits', overwrite=True)


Define and call huge plotting function

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.table import Table
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

def load_and_prepare_data(fits_file_path):
    """
    Load the FITS table and prepare data for comparison
    """
    # Load the FITS table
    table = Table.read(fits_file_path)
    
    # Extract F770W data from multi-dimensional arrays
    # For Flux and Flux_Err, take the first value (F770W filter)
    flux_personal = []
    flux_err_personal = []
    ab_mag_personal = []
    
    for i in range(len(table)):
        # Handle Flux (F770W is first element)
        if table['Flux'].mask[i][0] == False:
            flux_val = table['Flux'][i]*1e6 # convert to µJy
            if hasattr(flux_val, '__len__') and len(flux_val) > 0:
                flux_personal.append(flux_val[0])
            else:
                flux_personal.append(flux_val)
        else:
            flux_personal.append(np.nan)
            
        # Handle Flux_Err (F770W is first element)
        if table['Flux_Err'].mask[i][0] == False:
            flux_err_val = table['Flux_Err'][i]*1e6 # convert to µJy
            if hasattr(flux_err_val, '__len__') and len(flux_err_val) > 0:
                flux_err_personal.append(flux_err_val[0])
            else:
                flux_err_personal.append(flux_err_val)
        else:
            flux_err_personal.append(np.nan)
            
        # Handle AB_Mag (F770W is first element)
        if table['AB_Mag'].mask[i][0] == False:
            ab_mag_val = table['AB_Mag'][i]
            if hasattr(ab_mag_val, '__len__') and len(ab_mag_val) > 0:
                ab_mag_personal.append(ab_mag_val[0])
            else:
                ab_mag_personal.append(ab_mag_val)
        else:
            ab_mag_personal.append(np.nan)
    
    # Convert to numpy arrays
    flux_personal = np.array(flux_personal)
    flux_err_personal = np.array(flux_err_personal)
    ab_mag_personal = np.array(ab_mag_personal)
    
    # Extract public catalogue data
    flux_public = np.array(table['flux_auto_f770w'])
    flux_err_public = np.array(table['flux_err_auto_f770w'])
    mag_public = np.array(table['mag_auto_f770w'])
    
    # Create masks for valid data
    valid_flux = (~np.isnan(flux_personal)) & (~np.isnan(flux_public)) & \
                 (flux_personal > 0) & (flux_public > 0)
    valid_flux_err = (~np.isnan(flux_personal)) & (~np.isnan(flux_public)) & \
                     (flux_personal > 0) & (flux_public > 0)
    
    return {
        'table': table,
        'flux_personal': flux_personal,
        'flux_err_personal': flux_err_personal,
        'ab_mag_personal': ab_mag_personal,
        'flux_public': flux_public,
        'flux_err_public': flux_err_public,
        'mag_public': mag_public,
        'valid_flux': valid_flux,
        'valid_flux_err': valid_flux_err
    }

def calculate_statistics(x, y, valid_mask):
    """
    Calculate comparison statistics
    """
    if np.sum(valid_mask) < 3:
        return {}
    
    x_valid = x[valid_mask]
    y_valid = y[valid_mask]
    
    # Linear correlation
    corr_coef, p_value = stats.pearsonr(x_valid, y_valid)
    
    # Calculate residuals and statistics
    residuals = y_valid - x_valid
    mean_residual = np.mean(residuals)
    median_residual = np.median(residuals)
    std_residual = np.std(residuals)
    rms_residual = np.sqrt(np.mean(residuals**2))
    
    # Fractional differences for positive values
    frac_diff = (y_valid - x_valid) / x_valid
    median_frac_diff = np.median(frac_diff)
    mean_frac_diff = np.mean(frac_diff)
    std_frac_diff = np.std(frac_diff)
    
    return {
        'correlation': corr_coef,
        'p_value': p_value,
        'median_residual': median_residual,
        'mean_residual': mean_residual,
        'std_residual': std_residual,
        'rms_residual': rms_residual,
        'median_frac_diff': median_frac_diff,
        'mean_frac_diff': mean_frac_diff,
        'std_frac_diff': std_frac_diff,
        'n_objects': len(x_valid)
    }

def create_comparison_plot(data):
    """
    Create comprehensive comparison plots
    """
    fig = plt.figure(figsize=(15, 15))
    
    # Define color scheme
    colors = {'scatter': '#1f77b4', 'line': '#ff7f0e', 'hist': '#2ca02c'}
    
    # 1. Flux comparison (log-log scale)
    ax1 = plt.subplot(3, 3, 1)
    valid_flux = data['valid_flux']
    if np.sum(valid_flux) > 0:
        x_flux = data['flux_personal'][valid_flux]
        y_flux = data['flux_public'][valid_flux]
        
        ax1.scatter(x_flux, y_flux, alpha=0.6, s=30, color=colors['scatter'])
        
        # Add 1:1 line
        min_val = min(np.min(x_flux), np.min(y_flux))
        max_val = max(np.max(x_flux), np.max(y_flux))
        ax1.plot([min_val, max_val], [min_val, max_val], '--', color=colors['line'], lw=2)
        
        ax1.set_xscale('log')
        ax1.set_yscale('log')
        ax1.set_xlabel('Personal Catalogue Flux (µJy)')
        ax1.set_ylabel('COSMOS-Web DR1 Catalogue Flux (µJy)')
        ax1.set_title('Flux Comparison (F770W)')
        ax1.grid(True, alpha=0.3)
        
        # Add statistics
        stats_flux = calculate_statistics(x_flux, y_flux, np.ones(len(x_flux), dtype=bool))
        if stats_flux:
            ax1.text(0.05, 0.95, f'r = {stats_flux["correlation"]:.3f}\nN = {stats_flux["n_objects"]}', 
                    transform=ax1.transAxes, verticalalignment='top', 
                    bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    # 2. Flux error comparison (log-log scale)
    ax2 = plt.subplot(3, 3, 2)
    valid_flux_err = data['valid_flux_err']
    if np.sum(valid_flux_err) > 0:
        x_err = data['flux_err_personal'][valid_flux_err]
        y_err = data['flux_err_public'][valid_flux_err]
        
        ax2.scatter(x_err, y_err, alpha=0.6, s=30, color=colors['scatter'])
        
        # Add 1:1 line
        min_val = min(np.min(x_err), np.min(y_err))
        max_val = max(np.max(x_err), np.max(y_err))
        ax2.plot([min_val, max_val], [min_val, max_val], '--', color=colors['line'], lw=2)
        
        ax2.set_xscale('log')
        ax2.set_yscale('log')
        ax2.set_xlabel('Personal Catalogue Flux Error (µJy)')
        ax2.set_ylabel('COSMOS-Web DR1 Catalogue Flux Error (µJy)')
        ax2.set_title('Flux Error Comparison (F770W)')
        ax2.grid(True, alpha=0.3)
        
        # Add statistics
        stats_err = calculate_statistics(x_err, y_err, np.ones(len(x_err), dtype=bool))
        if stats_err:
            ax2.text(0.05, 0.95, f'r = {stats_err["correlation"]:.3f}\nN = {stats_err["n_objects"]}', 
                    transform=ax2.transAxes, verticalalignment='top',
                    bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    
    # 3. Signal-to-noise ratio comparison
    ax3 = plt.subplot(3, 3, 3)
    valid_snr = data['valid_flux'] & data['valid_flux_err']
    if np.sum(valid_snr) > 0:
        snr_personal = data['flux_personal'][valid_snr] / data['flux_err_personal'][valid_snr]
        snr_public = data['flux_public'][valid_snr] / data['flux_err_public'][valid_snr]
        
        max_idx = np.argmax(snr_public)
        
        snr_personal = np.delete(snr_personal, max_idx)
        snr_public = np.delete(snr_public, max_idx)
        
        ax3.scatter(snr_personal, snr_public, alpha=0.6, s=30, color=colors['scatter'])
        
        # Add 1:1 line
        min_val = min(np.min(snr_personal), np.min(snr_public))
        max_val = max(np.max(snr_personal), np.max(snr_public))
        ax3.plot([min_val, max_val], [min_val, max_val], '--', color=colors['line'], lw=2)
        
        ax3.set_xlabel('Personal Catalogue S/N')
        ax3.set_ylabel('COSMOS-Web DR1 Catalogue S/N')
        ax3.set_title('Signal-to-Noise Ratio Comparison')
        ax3.grid(True, alpha=0.3)
        
        # Add statistics
        stats_snr = calculate_statistics(snr_personal, snr_public, np.ones(len(snr_personal), dtype=bool))
        if stats_snr:
            ax3.text(0.05, 0.95, f'r = {stats_snr["correlation"]:.3f}\nN = {stats_snr["n_objects"]}', 
                    transform=ax3.transAxes, verticalalignment='top',
                    bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    # 4. Flux residuals vs flux
    ax4 = plt.subplot(3, 3, 4)
    if np.sum(valid_flux) > 0:
        x_flux = data['flux_personal'][valid_flux]
        y_flux = data['flux_public'][valid_flux]
        residuals = y_flux - x_flux
        
        ax4.scatter(x_flux, residuals, alpha=0.6, s=30, color=colors['scatter'])
        ax4.axhline(y=0, color=colors['line'], linestyle='--', lw=2)
        ax4.set_xscale('log')
        ax4.set_xlabel('Personal Catalogue Flux (µJy)')
        ax4.set_ylabel('Flux Residuals (Public - Personal)')
        ax4.set_title('Flux Residuals vs Flux')
        ax4.grid(True, alpha=0.3)
    
    # 5. Fractional flux differences
    ax5 = plt.subplot(3, 3, 5)
    if np.sum(valid_flux) > 0:
        x_flux = data['flux_personal'][valid_flux]
        y_flux = data['flux_public'][valid_flux]
        frac_diff = (y_flux - x_flux) / x_flux
        
        ax5.scatter(x_flux, frac_diff, alpha=0.6, s=30, color=colors['scatter'])
        ax5.axhline(y=0, color=colors['line'], linestyle='--', lw=2)
        ax5.set_xscale('log')
        ax5.set_xlabel('Personal Catalogue Flux (µJy)')
        ax5.set_ylabel('Fractional Flux Difference')
        ax5.set_title('Fractional Flux Differences')
        ax5.grid(True, alpha=0.3)
    
    # 6. Error bar comparison
    ax6 = plt.subplot(3, 3, 6)
    if np.sum(valid_flux) & np.sum(valid_flux_err) > 0:
        valid_both = valid_flux & valid_flux_err
        x_flux = data['flux_personal'][valid_both]
        y_flux = data['flux_public'][valid_both]
        x_err = data['flux_err_personal'][valid_both]
        y_err = data['flux_err_public'][valid_both]
        
        # Plot a subset of points with error bars to avoid cluttering
        #n_plot = min(50, len(x_flux))
        #indices = np.random.choice(len(x_flux), n_plot, replace=False)
        
        ax6.errorbar(x_flux, y_flux, 
                    xerr=x_err, yerr=y_err,
                    fmt='o', alpha=0.6, capsize=3, color=colors['scatter'])
        
        # Add 1:1 line
        min_val = min(np.min(x_flux), np.min(y_flux))
        max_val = max(np.max(x_flux), np.max(y_flux))
        ax6.plot([min_val, max_val], [min_val, max_val], '--', color=colors['line'], lw=2)
        
        ax6.loglog()
        ax6.set_xlabel('Personal Catalogue Flux (µJy)')
        ax6.set_ylabel('COSMOS-Web DR1 Catalogue Flux (µJy)')
        ax6.set_title(f'Flux with Error Bars')
        ax6.grid(True, alpha=0.3)
    
    # 7. Flux histogram comparison
    ax7 = plt.subplot(3, 3, 7)
    if np.sum(valid_flux) > 0:
        x_flux = data['flux_personal'][valid_flux]
        y_flux = data['flux_public'][valid_flux]
        
        bins = np.logspace(np.log10(min(np.min(x_flux), np.min(y_flux))), 
                          np.log10(max(np.max(x_flux), np.max(y_flux))), 20)
        
        ax7.hist(x_flux, bins=bins, alpha=0.6, label='Personal', color=colors['scatter'])
        ax7.hist(y_flux, bins=bins, alpha=0.6, label='COSMOS-Web DR1', color=colors['line'])
        ax7.set_xscale('log')
        ax7.set_xlabel('Flux (µJy)')
        ax7.set_ylabel('Number of Objects')
        ax7.set_title('Flux Distribution Comparison')
        ax7.legend()
        ax7.grid(True, alpha=0.3)
    
    # 8. Residuals histogram
    ax8 = plt.subplot(3, 3, 8)
    if np.sum(valid_flux) > 0:
        x_flux = data['flux_personal'][valid_flux]
        y_flux = data['flux_public'][valid_flux]
        frac_diff = (y_flux - x_flux) / x_flux
        
        ax8.hist(frac_diff, bins=30, alpha=0.7, color=colors['hist'])
        ax8.axvline(x=0, color=colors['line'], linestyle='--', lw=2)
        ax8.axvline(x=np.median(frac_diff), color='red', linestyle='-', lw=2, label=f'Median: {np.median(frac_diff):.3f}')
        ax8.axvline(x=np.mean(frac_diff), color='blue', linestyle='-', lw=2, label=f'Mean: {np.mean(frac_diff):.3f}')
        ax8.set_xlabel('Fractional Flux Difference')
        ax8.set_ylabel('Number of Objects')
        ax8.set_title('Fractional Difference Distribution')
        ax8.legend()
        ax8.grid(True, alpha=0.3)
    
    # 9. Summary statistics text
    ax12 = plt.subplot(3, 3, 9)
    ax12.axis('off')
    
    summary_text = "\n  PHOTOMETRY COMPARISON SUMMARY  \n\n"
    
    if np.sum(valid_flux) > 0:
        stats_flux = calculate_statistics(data['flux_personal'][valid_flux], 
                                        data['flux_public'][valid_flux], 
                                        np.ones(np.sum(valid_flux), dtype=bool))
        if stats_flux:
            summary_text += f"  FLUX COMPARISON (N={stats_flux['n_objects']}):  \n"
            summary_text += f"    Correlation: {stats_flux['correlation']:.3f}  \n"
            summary_text += f"    Median fractional diff: {stats_flux['median_frac_diff']:.3f}  \n"
            summary_text += f"    Mean fractional diff: {stats_flux['mean_frac_diff']:.3f}  \n"
            summary_text += f"    Std fractional diff: {stats_flux['std_frac_diff']:.3f}  \n\n"
    
    summary_text += f"  DATA COVERAGE:\n"
    summary_text += f"    Total objects: {len(data['table'])}  \n"
    summary_text += f"    Valid flux measurements: {np.sum(valid_flux)}  \n"
    
    ax12.text(0.05, 0.95, summary_text, transform=ax12.transAxes, 
             verticalalignment='top', fontfamily='monospace', fontsize=14,
             bbox=dict(boxstyle='round', facecolor='lightgray', alpha=0.8))
    
    plt.tight_layout()
    
    return fig

# Main execution function
def analyse_photometry_comparison(fits_file_path, output_plot_path=None):
    """
    Main function to perform photometry comparison analysis
    """
    print("Loading and preparing data...")
    data = load_and_prepare_data(fits_file_path)
    
    print("Creating comparison plots...")
    fig = create_comparison_plot(data)
    
    if output_plot_path:
        print(f"Saving plot to {output_plot_path}")
        plt.savefig(output_plot_path, dpi=300, bbox_inches='tight')
    
    plt.show()
    
    # Print summary statistics
    print("\n" + "="*60)
    print("PHOTOMETRY COMPARISON ANALYSIS SUMMARY")
    print("="*60)
    
    valid_flux = data['valid_flux']
    
    if np.sum(valid_flux) > 0:
        stats_flux = calculate_statistics(data['flux_personal'][valid_flux], 
                                        data['flux_public'][valid_flux], 
                                        np.ones(np.sum(valid_flux), dtype=bool))
        if stats_flux:
            print(f"\nFLUX COMPARISON ({stats_flux['n_objects']} objects):")
            print(f"  Pearson correlation coefficient: {stats_flux['correlation']:.4f}")
            print(f"  Mean fractional difference: {stats_flux['mean_frac_diff']:.4f}")
            print(f"  Standard deviation of fractional differences: {stats_flux['std_frac_diff']:.4f}")
            print(f"  RMS of absolute residuals: {stats_flux['rms_residual']:.4f} µJy")
    
    print(f"\nDATA COVERAGE:")
    print(f"  Total objects in table: {len(data['table'])}")
    print(f"  Objects with valid flux measurements: {np.sum(valid_flux)}")
    
    return data, fig


In [ ]:
 # Replace with your FITS file path
fits_file_path = '/home/bpc/University/master/Red_Cardinal/catalogues/COSMOSWeb_sky_matched.fits'


# Optional: specify output path for the plot
output_plot_path = '/home/bpc/University/master/Red_Cardinal/photometry/photometry_comparisons/phot_comp_sky_matched_new.png'

# Run the analysis
data, fig = analyse_photometry_comparison(fits_file_path, output_plot_path)

# Section to perform photometry with all 4 filters (F770W, F1000W, F1800W, F2100W)

Performing the photometry became much easier now!

In [ ]:

phot.perform_photometry(f770w_files, aperture_table, output_folder, suffix='v5')

phot.perform_photometry(f1000w_files, aperture_table, output_folder, suffix='v5')

phot.perform_photometry(f1800w_files, aperture_table, output_folder, suffix='v5')

phot.perform_photometry(f2100w_files, aperture_table, output_folder, suffix='v5')


Now combine the csv files into one big table

In [ ]:
fits_table_v5 = 'Flux_Aperture_PSFMatched_AperCorr_MIRI_v5.fits'

results_dir = '/home/bpc/University/master/Red_Cardinal/photometry/results/'

f770w_fname  = os.path.join(results_dir, 'phot_table_F770W_v5.csv')
f1000w_fname = os.path.join(results_dir, 'phot_table_F1000W_v5.csv')
f1800w_fname = os.path.join(results_dir, 'phot_table_F1800W_v5.csv')
f2100w_fname = os.path.join(results_dir, 'phot_table_F2100W_v5.csv')

csv_paths = [f770w_fname, f1000w_fname, f1800w_fname, f2100w_fname]

# Now create the combined FITS table
phot.create_fits_table_from_csv(csv_paths, output_file=fits_table_v5)

In [ ]:
# Example of how to read and interpret the flags:
fits_table_v5 = '/home/bpc/University/master/Red_Cardinal/photometry/phot_tables/Flux_Aperture_PSFMatched_AperCorr_MIRI_v5.fits'
table = Table.read(fits_table_v5)

for i, gid in enumerate(table['ID']):
    filters_available = table['Filters'][i].split(',')
    has_companion = table['Flag_Com'][i]
    artifact_flags = table['Flag_Art'][i]
    
    print(f"Galaxy {gid}: Companion = {has_companion}")
    for j, filt in enumerate(filters_available):
        has_artifact = artifact_flags[j]
        print(f"  {filt}: Artifact = {has_artifact}")
    print('\n')

In [ ]:
fits_table_v5 = '/home/bpc/University/master/Red_Cardinal/photometry/phot_tables/Flux_Aperture_PSFMatched_AperCorr_MIRI_v5.fits'
fig_path = '/home/bpc/University/master/Red_Cardinal/photometry/miri_coverage_v5.png'
stats_path = '/home/bpc/University/master/Red_Cardinal/photometry/galaxy_stats_v5.txt'
filter_to_ids = phot.galaxy_statistics(fits_table_v5, fig_path, stats_path)

ids_in_f770w = filter_to_ids.get('F70W', set())
ids_in_f1000w = filter_to_ids.get('F1000W', set())
ids_in_f1800w = filter_to_ids.get('F1800W', set())
ids_in_f2100w = filter_to_ids.get('F2100W', set())

Test for my newest function

In [ ]:
vis_dir = '/home/bpc/University/master/Red_Cardinal/photometry/vis_data'
mosaic_dir = '/home/bpc/University/master/Red_Cardinal/photometry/mosaic_plots/'
plane_sub_dir = '/home/bpc/University/master/Red_Cardinal/photometry/plane_sub/'

#phot.create_mosaics(vis_dir, mosaic_dir, plane_sub_dir)
phot.create_mosaics(vis_dir, plane_sub_dir)